In [ ]:
# !pip install openai-whisper pyannote-audio openai torch torchaudio librosa
# !pip install python-dotenv
# pip install ffmpeg
# apt install ffmpeg


Name: torch
Version: 2.7.1

- pip install --upgrade torchvision
Name: torchvision
Version: 0.22.1

- pip install protobuf==3.20.*
Name: protobuf
Version: 3.20.3

Hugging Face 계정 필요: https://huggingface.co

Hugging Face token 발급 후 사용

OpenAI GPT API 키 필요: https://platform.openai.com

👉 https://huggingface.co/pyannote/speaker-diarization 에 방문
👉 "Access Repository" 버튼 클릭해서 조건에 동의
pyannote/segmentation도 Access Repository 동의해야 함.


export HUGGINGFACE_TOKEN="hf_..."        # Hugging Face 토큰

export OPENAI_API_KEY="sk-..."           # OpenAI API 키

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경변수 읽기
HUGGINGFACE_USER = os.getenv("HUGGINGFACE_USER")
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(HUGGINGFACE_USER[:8] + "...")
print(HUGGINGFACE_TOKEN[:8] + "...")
print(OPENAI_API_KEY[:12] + "...")

moya92h...
hf_iSkSB...
sk-proj-fUgG...


In [1]:
import torchvision
print(torchvision.__file__)
print(torchvision.__version__)

/opt/conda/lib/python3.9/site-packages/torchvision/__init__.py
0.22.1+cu126


In [4]:
import os
import whisper
import openai
from pyannote.audio import Pipeline
import contextlib
import wave

openai.api_key = OPENAI_API_KEY

AUDIO_FILE = "meeting.wav"

# 1. Whisper: 음성 → 텍스트
print("🌀 Loading Whisper...")
whisper_model = whisper.load_model("base")
transcription = whisper_model.transcribe(AUDIO_FILE, verbose=False)
segments = transcription['segments']


# 2. pyannote: 화자 분리
print("🔍 Running speaker diarization...")
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=HUGGINGFACE_TOKEN)

print(type(pipeline))
print(pipeline)

diarization = pipeline(AUDIO_FILE)


# 3. 화자별 텍스트 정렬
speaker_texts = []

for segment in segments:
    start = segment['start']
    end = segment['end']
    text = segment['text'].strip()

    matched_speaker = None
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        if turn.start <= start <= turn.end or turn.start <= end <= turn.end:
            matched_speaker = speaker
            break
    speaker_texts.append((matched_speaker or "Unknown", text))

    
# 4. 전체 화자별 텍스트 합치기
combined_text = ""
for speaker, text in speaker_texts:
    combined_text += f"{speaker}: {text}\n"

print("\n🧾 화자 구분 텍스트:\n")
print(combined_text[:1000], "...")  # 길어서 앞부분만 출력

🌀 Loading Whisper...
Detected language: Korean


100%|██████████| 1041/1041 [00:00<00:00, 1781.02frames/s]


🔍 Running speaker diarization...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../aiffel/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1+cu126. Bad things might happen unless you revert torch to 1.x.
<class 'pyannote.audio.pipelines.speaker_diarization.SpeakerDiarization'>

🧾 화자 구분 텍스트:

SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠고요.
SPEAKER_00: 그리고 마지막으로 한 가지만 다요.
SPEAKER_00: 개시판에 한번 들러보시는 것도 좋을 것 같아요.
SPEAKER_00: 개시판에 가보면
 ...


from pyannote.audio import Model

model = Model.from_pretrained(
    "pyannote/segmentation",
    use_auth_token=HUGGINGFACE_TOKEN
)
print(model)

🔍 Running speaker diarization...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../aiffel/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1+cu126. Bad things might happen unless you revert torch to 1.x.


/opt/conda/lib/python3.9/site-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


<class 'pyannote.audio.pipelines.speaker_diarization.SpeakerDiarization'>



🧾 화자 구분 텍스트:

SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠고요.
SPEAKER_00: 그리고 마지막으로 한 가지만 다요.
SPEAKER_00: 개시판에 한번 들러보시는 것도 좋을 것 같아요.
SPEAKER_00: 개시판에 가보면
 ...


In [12]:
from transformers import pipeline

# 사전학습된 한국어/영어 요약모델을 쓸 수 있음
# (영어 모델이 대표적이며, 한국어는 mBART, KoBART 등도 있음)
#summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summarizer = pipeline("summarization", model="hyunwoongko/kobart")


def summarize_with_hf(transcript):
    print("🤖 Running summarization using Hugging Face model...")
    # 모델 입력 길이 제한 주의 (1024 토큰 이하 권장)
    max_chunk = 1000
    # 긴 텍스트면 여러 조각으로 나눠서 요약 후 합치기 (선택사항)
    inputs = [transcript[i:i+max_chunk] for i in range(0, len(transcript), max_chunk)]

    summaries = []
    for chunk in inputs:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    return " ".join(summaries)


# 사용 예시
summary = summarize_with_hf(combined_text)
print(summary)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Your max_length is set to 150, but you input_length is only 77. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


🤖 Running summarization using Hugging Face model...

 SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠고요.
SPEAKER_00: 그리고 마지막으로 한 가지만 다요.
SPEAKER_00: 그리고 마지막으로 한 가지만 다요.
SPEAKER_00: 개시판에 한번 들러보시는 것도 좋을 것 같아요.
SPEAKER_00: 개시판에 한번 들러보시는 것도 좋을 것 같아요.










 SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠고요.
SPEAKER_00: 그런 것들은 뭐 유료


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 👉 Hugging Face 모델로 요약
def summarize_with_hf(text):
    model_name = "gogamza/kobart-summarization"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HUGGINGFACE_TOKEN)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=HUGGINGFACE_TOKEN)

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=256, min_length=30, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 사용 예시
summary = summarize_with_hf(combined_text)
print(summary)

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/172k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Summarize: SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠여행여행여행여행여행여행여행여행성성SPPPEAKER_00: 그리고 마지막으로 한 가지만 다마리SPPPAKER_00: 개시판에 한번 들러보시는 것도 좋을 것이라며   SPEAKER_00: 개시판에 가보면 브SPEAKER_00: 개시판에 가보면 브SPEAKER_00: 개시판에 가보면 브SPEAKER_00: 개시판에 가보면 브SPEAKER_00: 개시판에 가보면 브SPEAKER_00: 개시판에 가보면 가보면 브SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있고 아닌 것도 있으니까 그런 것들은 유료행 것도 있고 아닌 것도 있으니까 그런 거를 좀 잘 활용하시면 좋겠여행여행여행여행여행여행여행여행여행여행여행여행여행여행여행하고 마지막으로 한 가지만 다 같이 한 가지만 다마리SPPPPPPPPEKER_00: 개시판에 한번 들러보


In [9]:
transcript = """
회의록:
- 김철수: 오늘 회의의 주요 안건은 예산 조정입니다.
- 이영희: 예산을 10% 삭감하는 것이 필요합니다.
- 박민수: 삭감된 예산은 마케팅에 재배분해야 합니다.
- 김철수: 마케팅 예산을 늘리는 것에 동의합니다.
"""

summary = summarize_with_hf(transcript)
print("회의 요약:", summary)

회의 요약: Summarize: summarize:  summarize:  summarize:   summarize:   summarize:   summarize:   summarize:  summarize:   summarize: 마케팅 예산을 늘리는 것에 동의하며 마케팅 예산을 늘리는 것에 동의하며 김철수: 마케팅 예산을 늘리는 것에 동의하며 마케팅 예산을 늘리는 것에 동의하며 마케팅 예산을 늘리는 것에 동의하는             ummarize:    summarize:    summarize:   summarize:                  예산 조정 안건은 예산 조정예산 조정팀은 예산 조정팀은 예산 조정자문- 이영희: 예산을 10% 삭감하는 것이 필요해지고  박민수: 삭감된 예산은 삭감된 예산은 삭감된 예산은 삭감된 예산은 마케팅에 재배분해야  김철수: 마케팅 예산을 늘리는 것에 동의


In [9]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# KoBART 요약 모델 로드
model_name = "digit82/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def summarize_with_kobart(text):
    print("📝 Hugging Face KoBART로 요약 중...")

    # 입력 토큰화
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)

    # 모델로 요약 생성
    summary_ids = model.generate(
        inputs,
        max_length=256,
        min_length=32,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# 사용 예시
summary = summarize_with_hf(combined_text)
print(summary)

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

🤖 Running summarization using Hugging Face model...
SPEAKER_00:    ‘ 활용하’   ‘  ’’ ‘”  ’‘ ’ ’  ” ‘ ‘.


In [5]:
#### 5. GPT 요약 요청
from openai import OpenAI

client = OpenAI()

def summarize_with_gpt(transcript):
    print("📩 Sending to GPT for summarization...")
    response = client.chat.completions.create(
        model="gpt-4.1",  # 또는 gpt-3.5-turbo
        messages=[
            {
                "role": "system",
                "content": "당신은 회의록 정리 도우미입니다. 대화 내용을 요약해 주세요. 화자 구분이 포함되어 있습니다."
            },
            {
                "role": "user",
                "content": f"""다음은 화자별로 정리된 회의 내용입니다. 주요 논의사항, 의사결정, 액션 아이템을 요약해 주세요:\n\n{transcript}"""
            }
        ],
        temperature=0.5,
        max_tokens=800,
    )
    return response.choices[0].message.content
   

summary = summarize_with_gpt(combined_text)

# 6. 결과 출력
print("\n✅ 요약 결과:\n")
print(summary)

📩 Sending to GPT for summarization...

✅ 요약 결과:

주요 논의사항:
- 유료 및 무료로 활용할 수 있는 다양한 자료/도구가 있으니 적절히 활용할 것을 권장함(SPEAKER_00).
- 개시판(게시판) 방문을 추천하며, 게시판 활용의 필요성 언급(SPEAKER_00).

의사결정:
- 별도의 명확한 의사결정 내용은 없음.

액션 아이템:
- 참여자들이 유료/무료 자료 및 도구를 적극적으로 활용할 것.
- 게시판에 방문하여 정보를 확인할 것.



🧾 화자 구분 텍스트:

SPEAKER_00: 그런 것들은 뭐 유료행 것도 있고 아닌 것도 있으니까 그런 거 좀 잘 활용하시면 좋겠고요.
SPEAKER_00: 그리고 마지막으로 한 가지만 다요.
SPEAKER_00: 개시판에 한번 들러보시는 것도 좋을 것 같아요.
SPEAKER_00: 개시판에 가보면



✅ 요약 결과:

주요 논의사항:
- 유료 및 무료로 활용할 수 있는 다양한 자료/도구가 있으니 적절히 활용할 것을 권장함(SPEAKER_00).
- 개시판(게시판) 방문을 추천하며, 게시판 활용의 필요성 언급(SPEAKER_00).

의사결정:
- 별도의 명확한 의사결정 내용은 없음.

액션 아이템:
- 참여자들이 유료/무료 자료 및 도구를 적극적으로 활용할 것.
- 게시판에 방문하여 정보를 확인할 것.

In [11]:
transcript = """
회의 시작하겠습니다. 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠습니다.
김 팀장님, 현재 개발 진행 상황 공유해 주세요.
"""

summary = summarize_with_kobart(transcript)
print("📌 요약 결과:\n", summary)

📝 Hugging Face KoBART로 요약 중...
📌 요약 결과:
 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠고 오늘은 신규 서비스 론칭 일정과 마케팅 전략을 논의하겠다며 신규회의 시작하겠다.


In [ ]:
# streamlit_app.py
import streamlit as st

st.title("🎙️ 자동 회의 요약기")

uploaded_file = st.file_uploader("회의 음성 파일 업로드", type=["wav", "mp3"])
if uploaded_file:
    audio_path = "uploaded.wav"
    with open(audio_path, "wb") as f:
        f.write(uploaded_file.read())

    text = transcribe_audio(audio_path)
    segments = diarize_speakers(audio_path, os.getenv("HUGGINGFACE_TOKEN"))
    combined = assemble_transcript(text, segments)
    summary = summarize_with_hf(combined)

    st.subheader("📝 요약 결과")
    st.write(summary)
    

In [5]:
import os
import whisper
import openai
from pyannote.audio import Pipeline
import contextlib
import wave


openai.api_key = OPENAI_API_KEY

AUDIO_FILE = "meeting.wav"

# 1. Whisper: 음성 → 텍스트
print("🌀 Loading Whisper...")
whisper_model = whisper.load_model("base")
transcription = whisper_model.transcribe(AUDIO_FILE, verbose=False)
segments = transcription['segments']

# 2. pyannote: 화자 분리
print("🔍 Running speaker diarization...")
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=HUGGINGFACE_TOKEN)
diarization = pipeline(AUDIO_FILE)

# 3. 화자별 텍스트 정렬
speaker_texts = []

for segment in segments:
    start = segment['start']
    end = segment['end']
    text = segment['text'].strip()

    matched_speaker = None
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        if turn.start <= start <= turn.end or turn.start <= end <= turn.end:
            matched_speaker = speaker
            break
    speaker_texts.append((matched_speaker or "Unknown", text))

# 4. 전체 화자별 텍스트 합치기
combined_text = ""
for speaker, text in speaker_texts:
    combined_text += f"{speaker}: {text}\n"

print("\n🧾 화자 구분 텍스트:\n")
print(combined_text[:1000], "...")  # 길어서 앞부분만 출력

# 5. GPT 요약 요청
def summarize_with_gpt(transcript):
    print("📩 Sending to GPT for summarization...")
    response = openai.ChatCompletion.create(
        model="gpt-4",  # 또는 gpt-3.5-turbo
        messages=[
            {
                "role": "system",
                "content": "당신은 회의록 정리 도우미입니다. 대화 내용을 요약해 주세요. 화자 구분이 포함되어 있습니다."
            },
            {
                "role": "user",
                "content": f"""다음은 화자별로 정리된 회의 내용입니다. 주요 논의사항, 의사결정, 액션 아이템을 요약해 주세요:\n\n{transcript}"""
            }
        ],
        temperature=0.5,
        max_tokens=800,
    )
    return response["choices"][0]["message"]["content"]

summary = summarize_with_gpt(combined_text)

# 6. 결과 출력
print("\n✅ 요약 결과:\n")
print(summary)

🌀 Loading Whisper...


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'